In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split as tts
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_absolute_error, mean_squared_error, SCORERS
import scipy as sp
from joblib import dump,load
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.linear_model import Ridge, RidgeCV, LassoCV, ElasticNetCV, ElasticNet
from sklearn.model_selection import cross_val_score, cross_validate, GridSearchCV

# Logistic Regression

Is a classification algorithim designed to predict categorical data.

## Linear to logistic convertion

Having a dataset with two values such as True and False (1, 0), linear regression wouldnt work as we are going to have a large amount of outliers.

During logistic regression we are going to treat y as a probability plot of a value for x to be either a 0 or a 1, setting 0.5 as a cutoff.

To convert this function, we just have to plug our linear regression function into the logistic regression, but how do we relate out beta coeff with our results?

### Logistic Function

$$ \sigma (x) = \frac{1} {1+e^{-x}}  $$

### Linear Regression Function
$$
\hat{y} =  \beta_0 x_0 + ... + \beta_n x_n
$$

### Transformation

$$
\hat{y} = x 
$$
$$
\sigma (x) = \frac{1} {1+e^{-\sum_{i=0}^n \beta_i x_i}} 
$$
We can also write it like this.
$$
\hat{y} = \sigma \left({\sum_{i=0}^n \beta_i x_i}\right)
$$

For us to understand our predicted outputs we need to understand **odds**.

### Odds function

The odds of an event with probability **p** is defined by the chance of the event happening divided by the chance of it not happening.

$$
\frac{p}{1-p}
$$

Having and event with  $50$% of probability would follow the next odds:

$$
\frac{0.5}{1-0.5} = \frac{0.5}{0.5} = \frac{1}{1}
$$

Giving us 1 to 1 odds of it ocurring.

For our transformed function we can solve for the coeff and feature x in terms of the log odds. How do we saolve for the log odds?

$$
\hat{y} + \hat{y}e^{-\sum_{i=0}^n \beta_i x_i} = 1 
$$

We subtract $\hat{y}$ from both sides, resulting in:

$$
\hat{y}e^{-\sum_{i=0}^n \beta_i x_i} = 1 - \hat{y}
$$

Now we divide both sides by $\hat{y}$ and rearrange:

$$
\frac{1 - \hat{y}}{\hat{y}} = e^{-\sum_{i=0}^n \beta_i x_i}
$$

We are able to remove the constant $e$ from the equation by taking the natural logarithm of the function:

$$
ln\left( \frac{1 - \hat{y}}{\hat{y}} \right) = \sum_{i=0}^n \beta_i x_i
$$